# Example usage of Orca with XYZ files

You can still use molobjs (or ppqm functions) directly, if you have XYZ files to compute.


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import logging
import sys

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem.Draw import MolsToGridImage

In [ ]:
try:
    import ppqm
except ModuleNotFoundError:
    import pathlib

    cwd = pathlib.Path().resolve().parent
    sys.path.append(str(cwd))
    import ppqm

In [ ]:
from ppqm import jupyter as ppqm_jupyter

In [ ]:
import rmsd

## Set logging level

In [ ]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger("ppqm").setLevel(logging.INFO)
logging.getLogger("orca").setLevel(logging.DEBUG)
show_progress = True

## Load a XYZ file

In [ ]:
filename = "../tests/resources/compounds/methane.xyz"

In [ ]:
atoms, coordinates = rmsd.get_coordinates_xyz(filename)

In [ ]:
atoms

In [ ]:
charge = 0
spin = 1

## View the molecule

Note that no explict bonds are written in XYZ format, only the coordinates.
So RDKit does not now how the graph of the molecule looks like


In [ ]:
molobj = ppqm.chembridge.axyzc_to_molobj(atoms, coordinates, charge)
molobj

In [ ]:
ppqm_jupyter.show_molobj(molobj)

## Set Orca Settings

In [ ]:
orca_options = {
    "scr": "./_tmp_directory_",  # Where should the calculations happen?
    "cmd": "orca",  # What is the executable?
    "n_cores": 4,  # How many cores to use? 
    "memory": 1, # How much memory to use (per core)?
    "show_progress": show_progress
}

## Calculate some properties

We can now define calculation options and get Orca properties for each conformer in the molobj.


In [ ]:
# Optimize molecule in water
calculation_option = {
    "Freq": None,
    "B3LYP": None,
    "def2-SVP": None,
    "D3BJ": None,
    "Hirshfeld": None,
    "CPCM": "water",
    "RIJCOSX": None,
    "def2/J": None,
    "Grid4": None,
    "GridX4": None,
    "NMR": None,
    "def2/JK": None
}

Optimize molobj and set resulting coordinates. The `.calculate` function will return a `List[Dict]` with a property dictionary for each conformer.

In [ ]:
molecule_properties = ppqm.orca.get_properties_from_axyzc(
    atoms, coordinates, charge, spin, options=calculation_option, **orca_options
)

In [ ]:
# The output type
print(type(molecule_properties))
molecule_properties

In [ ]:
energy = molecule_properties["scf_energy"]
gibbs_free_energy = molecule_properties["gibbs_free_energy"]
print(f"Singlepoint calculation found for the provided geometry: {molecule_properties['stationary_points']}")
print(f"The energy was {energy:.2f} kcal/mol")
print(f"The Gibbs free energy was {gibbs_free_energy:.2f} kcal/mol")